<center><img src="https://keras.io/img/logo-small.png" alt="Keras logo" width="100"><br/>
This starter notebook is provided by the Keras team.</center>

# Getting Started on LLM Classification Fine Tuning with [KerasNLP](https://github.com/keras-team/keras-nlp) and [Keras](https://github.com/keras-team/keras)

<div align="center">
    <img src="https://i.ibb.co/wJMF5HL/lmsys.png">
</div>

In this competition, our aim is to predict which LLM responses users will prefer in a head-to-head battle between chatbots powered by large language models (LLMs). In other words, the goal of the competition is to predict the preferences of the judges and determine the likelihood that a given prompt/response pair is selected as the winner. This notebook will guide you through the process of fine-tuning the **DebertaV3** model for this competition using the **Shared Weight** strategy with KerasNLP. This strategy is similar to how Multiple Choice Question (MCQ) models are trained. Additionally, we will use mixed precision for faster training and inference.

**Did you know**: This notebook is backend-agnostic, which means it supports TensorFlow, PyTorch, and JAX backends. However, the best performance can be achieved with `JAX`. KerasNLP and Keras enable the choice of the preferred backend. Explore further details on [Keras](https://keras.io/keras_3/).

**Note**: For a deeper understanding of KerasNLP, refer to the [KerasNLP guides](https://keras.io/keras_nlp/).


# 📚 | Import Libraries 

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # or "tensorflow" or "torch"

import keras_nlp
import keras
import tensorflow as tf


import random
import numpy as np 
import pandas as pd
from tqdm import tqdm
import json

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
import wandb

# 將這行放在程式最前面，替換 YOUR_API_KEY 為你的實際 key
key="cfbd7f24413a77f911641acae8914b1dc77e1b83"

## Library Version

# ⚙️ | Configuration

In [ ]:
class CFG:
    seed = 42  # Random seed
    preset = "deberta_v3_extra_small_en" # Name of pretrained models
    sequence_length = 512  # Input sequence length
    epochs = 8 # Training epochs
    batch_size = 16  # Batch size

    # optimizer & scheduler
    learning_rate = 5e-6
    weight_decay = 1e-4
    scheduler = 'cosine'        # we’ll build a cosine decay schedule
    warmup_ratio = 0.1          # 用在 cosine warmup
    
    # regularization
    dropout_rate = 0.2          # head 裡的 Dropout

    # train / fine-tune 策略
    head_only_epochs = 3        # 前幾個 epoch 只訓練 head
    unfrozen_backbone_layers = 4 # 最後再解凍 backbone 後面幾層
    fine_tune_epochs = 5          # 解冻后再训的轮数
    
    # labels
    label2name = {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}
    name2label = {v:k for k, v in label2name.items()}
    class_labels = list(label2name.keys())
    class_names = list(label2name.values())

# ♻️ | Reproducibility 
Sets value for random seed to produce similar result in each run.

In [ ]:
tf.keras.utils.set_random_seed(CFG.seed)
np.random.seed(CFG.seed)
random.seed(CFG.seed)

# 🧮 | Mixed Precision

In this notebook, we will use mixed precision instead of float32 precision for training and inference to reduce GPU memory usage. This will ultimately allow us to use larger batch sizes, thus reducing our training and inference time.

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

# 📁 | Dataset Path 

In [ ]:
BASE_PATH = '/kaggle/input/llm-classification-finetuning'

# 📖 | Meta Data 

The competition dataset comprises user interactions from the ChatBot Arena. In each interaction, a judge presents one or more prompts to two different large language models and then indicates which model provided the more satisfactory response. The training data contains `55,000` rows, with an expected `25,000` rows in the test set.

## Files

### `train.csv`
- `id`: Unique identifier for each row.
- `model_[a/b]`: Model identity, present in train.csv but not in test.csv.
- `prompt`: Input prompt given to both models.
- `response_[a/b]`: Model_[a/b]'s response to the prompt.
- `winner_model_[a/b/tie]`: Binary columns indicating the judge's selection (ground truth target).

### `test.csv`
- `id`: Unique identifier for each row.
- `prompt`: Input prompt given to both models.
- `response_[a/b]`: Model_[a/b]'s response to the prompt.

> Note that each interaction may have multiple prompts and responses, but this notebook will use only **one prompt per interaction**. You can choose to use all prompts and responses. Additionally, prompts and responses in the dataframe are provided as string-formatted lists, so they need to be converted to literal lists using `eval()`.


## Train Data

In [ ]:

import json
from pathlib import Path

# 1. 建立路徑
BASE_PATH = Path(BASE_PATH)  # 如果你之前已經是 str，這行可把它轉成 Path
train_file = BASE_PATH / "train.csv"
if not train_file.exists():
    raise FileNotFoundError(f"找不到訓練檔案：{train_file}")

# 2. 讀檔並打散
df = pd.read_csv(train_file)
df = df.sample(frac=1, random_state=CFG.seed).reset_index(drop=True)

# 3. 安全地解析 JSON string 並取第一個元素
def safe_first_element(json_str: str):
    if not isinstance(json_str, str) or len(json_str) == 0:
        return ""
    # 把 'null' 換成 JSON 可接受的空字串
    cleaned = json_str.replace("null", '""')
    try:
        arr = json.loads(cleaned)
        if isinstance(arr, list) and len(arr) > 0:
            return arr[0]
    except json.JSONDecodeError:
        pass
    return ""

df["prompt"]     = df["prompt"].apply(safe_first_element)
df["response_a"] = df["response_a"].apply(safe_first_element)
df["response_b"] = df["response_b"].apply(safe_first_element)

# 4. 標籤轉換（和你原本一樣）
df["class_name"]  = df[["winner_model_a","winner_model_b","winner_tie"]].idxmax(axis=1)
df["class_label"] = df["class_name"].map(CFG.name2label)

# 5. 檢視結果
df.head()


In [ ]:

# 1. 計算缺失值
missing_values = df.isnull().sum()
missing_columns = missing_values[missing_values > 0]

print("缺失值統計：")
print(missing_values)

if not missing_columns.empty:
    print("\n有缺失值的欄位：")
    print(missing_columns)

# 2. 互動式顯示缺失值表格（可排序／篩選）
#from ace_tools import display_dataframe_to_user

missing_df = (
    missing_values
    .rename("missing_count")
    .to_frame()
    .assign(missing_pct=lambda d: d["missing_count"] / len(df) * 100)
    .reset_index()
    .rename(columns={"index": "column"})
    .query("missing_count > 0")
)

#display_dataframe_to_user("缺失值欄位一覽", missing_df)
print("\n缺失值欄位一覽：")
print(missing_df)

# 3. 處理缺失值
#    (a) 針對關鍵欄位直接丟棄含 NaN 的列
df = df.dropna(subset=["prompt", "response_a", "response_b"]).reset_index(drop=True)

#    (b) 其餘欄位用空字串填補
df = df.fillna("")

# 4. 最後檢視處理後的前幾筆
print(df.head())


## Test Data

In [ ]:

import json
from pathlib import Path

# 1. 確保 BASE_PATH 是 Path 物件
BASE_PATH = Path(BASE_PATH)
test_file = BASE_PATH / "test.csv"
if not test_file.exists():
    raise FileNotFoundError(f"找不到測試檔案：{test_file}")

# 2. 讀取測試資料
test_df = pd.read_csv(test_file)

# 3. 安全解析函式（同訓練集）
def safe_first_element(json_str: str):
    if not isinstance(json_str, str) or len(json_str) == 0:
        return ""
    cleaned = json_str.replace("null", '""')
    try:
        arr = json.loads(cleaned)
        if isinstance(arr, list) and len(arr) > 0:
            return arr[0]
    except json.JSONDecodeError:
        pass
    return ""

# 4. 處理 prompt / response 欄位
test_df["prompt"]     = test_df["prompt"].apply(safe_first_element)
test_df["response_a"] = test_df["response_a"].apply(safe_first_element)
test_df["response_b"] = test_df["response_b"].apply(safe_first_element)

# 5. 檢視前幾筆
print(test_df.head())


### Alec 查看資料狀態

In [ ]:
# 1. 計算 winner
winner = df[['winner_model_a','winner_model_b','winner_tie']].idxmax(axis=1).map({
    'winner_model_a':'A_win',
    'winner_model_b':'B_win',
    'winner_tie':'Tie'
})
# 2. 計算比例並排序
counts = winner.value_counts(normalize=True).sort_index()  

# 3. 繪圖
import matplotlib.pyplot as plt

plt.figure()
ax = counts.plot.bar(title='Winner Distribution')
ax.set_ylabel('Proportion')

# 4. 標註百分比
for patch in ax.patches:
    height = patch.get_height()
    ax.annotate(f"{height:.1%}",
                (patch.get_x() + patch.get_width() / 2, height),
                ha='center', va='bottom')

plt.show()

# 5. 印出數值
print(counts)

In [ ]:
pair_stats = (
    df
    .groupby(['model_a','model_b'])
    .agg(
        total=('id','size'),
        a_wins=('winner_model_a','sum'),
        b_wins=('winner_model_b','sum'),
        ties=('winner_tie','sum'),
    )
    .assign(
        a_win_rate=lambda d: (d['a_wins']/d['total']).round(3),
        b_win_rate=lambda d: (d['b_wins']/d['total']).round(3),
        tie_rate=lambda d: (d['ties']/d['total']).round(3),
    )
    .reset_index()
    .sort_values('total', ascending=False)
)

display(pair_stats.head(10))


## Contextualize Response with Prompt

In our approach, we will contextualize each response with the prompt instead of using a single prompt for all responses. This means that for each response, we will provide the model with the same set of prompts combined with their respective response (e.g., `(P + R_A)`, `(P + R_B)`, etc.). This approach is similar to the multiple-choice question task in NLP.

> Note that some prompts and responses may not be encoded with `utf-8`, resulting in errors when creating the dataloader. In such cases, we will replace them with an empty string.


In [ ]:
# Define a function to create options based on the prompt and choices
def make_pairs(row):
    def clean_text(text: any) -> str:
        """
        將輸入強制轉成 str，再丟掉非法 UTF-8 字元。
        """
        if not isinstance(text, str):
            return ""
        return text.encode("utf-8", errors="ignore") \
                   .decode("utf-8", errors="ignore")

    # 一次拿到乾淨的 prompt / responses
    prompt     = clean_text(row.get("prompt", ""))
    response_a = clean_text(row.get("response_a", ""))
    response_b = clean_text(row.get("response_b", ""))

    # 判斷是否有非字串輸入
    row["encode_fail"] = any(not isinstance(row.get(col, None), str)
                              for col in ["prompt", "response_a", "response_b"])

    # 建立 options
    row["options"] = [
        f"Prompt: {prompt}\n\nResponse A: {response_a}",
        f"Prompt: {prompt}\n\nResponse B: {response_b}"
    ]
    return row

In [ ]:
df = df.apply(make_pairs, axis=1)  # Apply the make_pairs function to each row in df
display(df.head(2))  # Display the first 2 rows of df

test_df = test_df.apply(make_pairs, axis=1)  # Apply the make_pairs function to each row in df
display(test_df.head(2))  # Display the first 2 rows of df

## Encoding Fail Statistics

Let's examine how many samples have encoding issues. From the code below, we can see that only $1\%$ of the samples failed to be encoded, while $99\%$ of the samples don't have any issues. A similar pattern can be expected for the test data as well. Thus, considering empty strings for this small portion of the data will not have much impact on our training and inference.

In [ ]:
df.encode_fail.value_counts(normalize=False)

# 🎨 | Exploratory Data Analysis (EDA)

## LLM Distribution

In [ ]:
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

In [ ]:
import plotly.express as px

# 1. 合併並計算次數
llm_series = pd.concat([df.model_a, df.model_b], ignore_index=True)
llm_counts = (
    llm_series
    .value_counts()                      # 得到一個 Series，index 是 LLM，值是出現次數
    .rename_axis("LLM")                  # 把 index 名稱設為 "LLM"
    .reset_index(name="Count")           # 把 Series 轉成 DataFrame，並把值列命名為 "Count"
)
# 2. 排序（降冪）
llm_counts = llm_counts.sort_values(by="Count", ascending=False)
print(llm_counts.columns)

# 3. 畫圖
plt.figure(figsize=(12, 6))
bars = plt.bar(llm_counts["LLM"], llm_counts["Count"])
for bar in bars:
    h = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        h,
        f"{h}",
        ha="center",
        va="bottom",
        fontsize=9
    )

plt.xticks(rotation=45, ha="right")

plt.title("Distribution of LLMs")
plt.xlabel("LLM")
plt.ylabel("Count")


plt.tight_layout()

plt.show()


## Winning Distribution

In [ ]:
# 計算並重構 counts DataFrame
counts = (
    df["class_name"]
    .value_counts()                      # Series: index=class_name, value=frequency
    .rename_axis("Winner")               # 把 index 名稱改成 Winner
    .reset_index(name="Win Count")       # 把 value 變成 Win Count 欄位
)
print("Columns in counts:", counts.columns.tolist())

counts_sorted = counts.sort_values(by="Win Count", ascending=False).reset_index(drop=True)

plt.figure(figsize=(8, 5))
bars = plt.bar(counts_sorted["Winner"], counts_sorted["Win Count"])


for bar in bars:
    h = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        h,
        f"{h}",
        ha="center",
        va="bottom",
        fontsize=9
    )

plt.xticks(rotation=45, ha="right")


plt.title("Winner Distribution for Train Data")
plt.xlabel("Winner")
plt.ylabel("Win Count")


plt.tight_layout()
plt.show()








# 🔪 | Data Split

In the code snippet provided below, we will divide the existing data into training and validation using a stratification of `class_label` column.

In [ ]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["class_label"],
    shuffle=True,
    random_state=CFG.seed,    # ← 加上這行，保證每次切分都一樣
)


# 🍽️ | Preprocessing

**What it does:** The preprocessor takes input strings and transforms them into a dictionary (`token_ids`, `padding_mask`) containing preprocessed tensors. This process starts with tokenization, where input strings are converted into sequences of token IDs.

**Why it's important:** Initially, raw text data is complex and challenging for modeling due to its high dimensionality. By converting text into a compact set of tokens, such as transforming `"The quick brown fox"` into `["the", "qu", "##ick", "br", "##own", "fox"]`, we simplify the data. Many models rely on special tokens and additional tensors to understand input. These tokens help divide input and identify padding, among other tasks. Making all sequences the same length through padding boosts computational efficiency, making subsequent steps smoother.

Explore the following pages to access the available preprocessing and tokenizer layers in **KerasNLP**:
- [Preprocessing](https://keras.io/api/keras_nlp/preprocessing_layers/)
- [Tokenizers](https://keras.io/api/keras_nlp/tokenizers/)

In [ ]:
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    preset=CFG.preset, # Name of the model
    sequence_length=CFG.sequence_length, # Max sequence length, will be padded if shorter
)

Now, let's examine what the output shape of the preprocessing layer looks like. The output shape of the layer can be represented as $(num\_responses, sequence\_length)$.

In [ ]:
outs = preprocessor(df.options.iloc[0])  # Process options for the first row

# Display the shape of each processed output
for k, v in outs.items():
    print(k, ":", v.shape)

We'll use the `preprocessing_fn` function to transform each text option using the `dataset.map(preprocessing_fn)` method.

In [ ]:
def preprocess_fn_train(options: list[str], class_label: int):
    """
    用在訓練/驗證：輸入一個 options list 和它的 class_label，
    回傳 (model_inputs_dict, class_label)。
    """
    # 安全檢查
    if not isinstance(options, list) or len(options) == 0:
        # full of zeros, 兩段 input_ids, attention_mask, etc.
        empty_inputs = {
            k: tf.zeros((len(options) or 1, CFG.sequence_length), dtype=tf.int32)
            for k in ["input_ids", "attention_mask", "token_type_ids"]
        }
        return empty_inputs, class_label

    # 真正 call preprocessor
    model_inputs = preprocessor(options)
    return model_inputs, class_label


def preprocess_fn_test(options: list[str]):
    """
    用在測試/推論：只有 options，回傳 model_inputs_dict。
    """
    if not isinstance(options, list) or len(options) == 0:
        return {
            k: tf.zeros((len(options) or 1, CFG.sequence_length), dtype=tf.int32)
            for k in ["input_ids", "attention_mask", "token_type_ids"]
        }
    return preprocessor(options)


# 🍚 | DataLoader

The code below sets up a robust data flow pipeline using `tf.data.Dataset` for data processing. Notable aspects of `tf.data` include its ability to simplify pipeline construction and represent components in sequences.

To learn more about `tf.data`, refer to this [documentation](https://www.tensorflow.org/guide/data).

In [ ]:
def build_dataset(options, labels=None, batch_size=32,
                  cache=True, shuffle_size=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNE option
     # 1. 建立原始 ds，並選擇對應的 map function
    if labels is not None:
        ds = tf.data.Dataset.from_tensor_slices((options, labels))
        map_fn = preprocess_fn_train
    else:
        ds = tf.data.Dataset.from_tensor_slices(options)
        map_fn = preprocess_fn_test

    # 2. Shuffle（只在有 labels 時，且 shuffle_size > 0）
    if labels is not None and shuffle_size > 0:
        ds = ds.shuffle(shuffle_size, seed=CFG.seed, reshuffle_each_iteration=True)

    # 3. Cache（可選）
    if cache:
        ds = ds.cache()

    # 4. Map → preprocessing
    ds = ds.map(map_fn, num_parallel_calls=AUTO)

    # 5. Batch & Prefetch
    ds = ds.batch(batch_size, drop_remainder=False)
    ds = ds.prefetch(AUTO)

    return ds

## Build Train/Valid Dataloader

In [ ]:
# Train
train_texts = train_df["options"].tolist()
train_labels = train_df["class_label"].tolist()
train_ds = build_dataset(
    train_texts,
    train_labels,
    batch_size=CFG.batch_size,
    cache=True,       # cache 訓練集以加速多 epoch 重跑
    shuffle_size=1024      # shuffle buffer size
)

# Valid
valid_texts = valid_df["options"].tolist()
valid_labels = valid_df["class_label"].tolist()
valid_ds = build_dataset(
    valid_texts,
    valid_labels,
    batch_size=CFG.batch_size,
    cache=False,      # 驗證集不需要 cache
    shuffle_size=0         # 不做 shuffle
)


# ⚓ | LR Schedule

Implementing a learning rate scheduler is crucial for transfer learning. The learning rate initiates at `lr_start` and gradually tapers down to `lr_min` using various techniques, including:
- `step`: Lowering the learning rate in step-wise manner resembling stairs.
- `cos`: Utilizing a cosine curve to gradually reduce the learning rate.
- `exp`: Exponentially decreasing the learning rate.

**Importance:** A well-structured learning rate schedule is essential for efficient model training, ensuring optimal convergence and avoiding issues such as overshooting or stagnation.

In [ ]:

def get_cosine_warmup_schedule(steps_per_epoch: int):
    total_steps   = CFG.epochs * steps_per_epoch
    warmup_steps  = int(total_steps * CFG.warmup_ratio)
    # 1. 先做 warmup：从 0 → learning_rate
    warmup_fn = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=0.0,
        decay_steps=warmup_steps,
        end_learning_rate=CFG.learning_rate,
        power=1.0
    )
    # 2. 再做剩余步数的 cosine decay：从 learning_rate → weight_decay（当作最小 lr）
    cosine_fn = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=CFG.learning_rate,
        decay_steps=total_steps - warmup_steps,
        alpha=CFG.weight_decay/CFG.learning_rate  # α = lr_min / lr_max
    )
    # 3. 用 `tf.where` 把两个 schedule 拼起来
    def lr_schedule_fn(step):
        return tf.where(
            step < warmup_steps,
            warmup_fn(step),
            cosine_fn(step - warmup_steps)
        )
    return lr_schedule_fn




In [ ]:

from tensorflow.keras.optimizers import AdamW


steps_per_epoch = len(train_df) // CFG.batch_size
lr_fn = get_cosine_warmup_schedule(steps_per_epoch)

# 使用內建的 AdamW
optimizer = AdamW(
    learning_rate=lr_fn,
    weight_decay=CFG.weight_decay
)


# 💾 | Model Checkpointing

The following code will create a callback that will save the best checkpoint of the model during training, which we will use for inference in the submission.

In [ ]:
# 把 checkpoint 存到專案的 checkpoints 目錄，先確保它存在
ckpt_dir = Path("checkpoints")
ckpt_dir.mkdir(exist_ok=True)

ckpt_cb = keras.callbacks.ModelCheckpoint(
    filepath=str(ckpt_dir / "best_model.{epoch:02d}-{val_loss:.4f}.weights.h5"),
    monitor="val_loss",             # 要對應 compile 時的 loss 名稱
    save_best_only=True,
    save_weights_only=True,          # 只存 weights
    mode="min",                      # loss 越低代表越好
    verbose=1                        # 儲存時顯示訊息
)

# 🤖 | Modeling

The `KerasNLP` library provides various NLP model architectures such as `Bert`, `Roberta`, `DebertaV3`, and more. While this notebook focuses on `DebertaV3`, you can explore others in the [KerasNLP documentation](https://keras.io/api/keras_nlp/models/). For a deeper understanding, refer to the [getting started guide](https://keras.io/guides/keras_nlp/getting_started/).

Our approach utilizes `keras_nlp.models.DebertaV3Classifier` to process each prompt and response pair, generating output embeddings. We then concatenate these embeddings and pass them through a Pooling layer and a classifier to obtain logits, followed by a `softmax` function for the final output.

When dealing with multiple responses, we use a weight-sharing strategy. This means we provide the model with one response at a time along with the prompt `(P + R_A)`, `(P + R_B)`, etc., using the same model weights for all responses. After obtaining embeddings for all responses, we concatenate them and apply average pooling. Next, we use a `Linear/Dense` layer along with the `Softmax` function as the classifier for the final result. Providing all responses at once would increase text length and complicate model handling. Note that, in the classifier, we use 3 classes for `winner_model_a`, `winner_model_b`, and `draw` cases.

The diagram below illustrates this approach:

<div align="center">
    <img src="https://i.postimg.cc/g0gcvy3f/Kaggle-drawio.png">
</div>

From a coding perspective, note that we use the same model for all responses with shared weights, contrary to the separate models implied in the diagram.

In [ ]:
# Define input layers

import math
from tensorflow.keras import mixed_precision, regularizers
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
# 1. Mixed‐precision
mixed_precision.set_global_policy("mixed_float16")

# 2. Define inputs matching preprocessor output keys
inputs = {
    "input_ids": keras.Input(
        shape=(2, CFG.sequence_length), dtype=tf.int32, name="input_ids"
    ),
    "attention_mask": keras.Input(
        shape=(2, CFG.sequence_length), dtype=tf.int32, name="attention_mask"
    ),
}

# 3. Backbone + freeze; expects 'token_ids' & 'padding_mask'
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(CFG.preset)
backbone.trainable = False

# 4. Slice & rename inputs for backbone
def slice_pair_and_rename(inputs_dict, idx):
    return {
        "token_ids":    inputs_dict["input_ids"][:, idx, :],
        "padding_mask": inputs_dict["attention_mask"][:, idx, :],
    }
# Backbone (frozen initially)
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(CFG.preset)
backbone.trainable = False

# Helper to slice the pair dimension and rename keys for the backbone
def slice_pair_and_rename(inputs_dict, idx):
    return {
        "token_ids":    inputs_dict["input_ids"][:, idx, :],
        "padding_mask": inputs_dict["attention_mask"][:, idx, :],
    }
embed_a = backbone(slice_pair_and_rename(inputs, 0))
embed_b = backbone(slice_pair_and_rename(inputs, 1))

# 5. Head
x = keras.layers.Concatenate(axis=1)([embed_a, embed_b])
x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(CFG.dropout_rate)(x)
outputs = keras.layers.Dense(
    len(CFG.class_names),
    activation="softmax",
    dtype="float32",
    kernel_regularizer=regularizers.l2(CFG.weight_decay),
    name="classifier",
)(x)

model = keras.Model(inputs, outputs)

# --- 6. Custom Cosine‐with‐Warmup Schedule ---
class CosineWarmupSchedule(LearningRateSchedule):
    def __init__(self, lr_max, lr_min, warmup_steps, total_steps):
        super().__init__()
        self.lr_max = lr_max
        self.lr_min = lr_min
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps

    def __call__(self, step):
        # convert to float tensors
        step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        total_steps = tf.cast(self.total_steps, tf.float32)

        # linear warmup: 0 -> lr_max
        warmup_lr = self.lr_max * (step / warmup_steps)

        # cosine decay: lr_max -> lr_min
        decay_step = step - warmup_steps
        decay_steps = total_steps - warmup_steps
        cosine_decay = 0.5 * (1 + tf.cos(math.pi * decay_step / decay_steps))
        decay_lr = self.lr_min + (self.lr_max - self.lr_min) * cosine_decay

        return tf.where(step < warmup_steps, warmup_lr, decay_lr)

    def get_config(self):
        return {
            "lr_max": self.lr_max,
            "lr_min": self.lr_min,
            "warmup_steps": self.warmup_steps,
            "total_steps": self.total_steps,
        }

# compute schedule parameters
steps_per_epoch = len(train_df) // CFG.batch_size
total_steps  = CFG.head_only_epochs * steps_per_epoch + CFG.fine_tune_epochs * steps_per_epoch
warmup_steps = int(CFG.head_only_epochs * steps_per_epoch * CFG.warmup_ratio)

lr_schedule = CosineWarmupSchedule(
    lr_max=CFG.learning_rate,
    lr_min=CFG.weight_decay,
    warmup_steps=warmup_steps,
    total_steps=total_steps,
)

# --- 7. Optimizer & Compile ---
# Stage 1: train head only
optimizer_stage1 = AdamW(
    learning_rate=lr_schedule,
    weight_decay=CFG.weight_decay,
)
model.compile(
    optimizer=optimizer_stage1,
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="accuracy")],
)
print(f" Stage1: training head for {CFG.head_only_epochs} epochs")
history_head = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=CFG.head_only_epochs,
    callbacks=[
        EarlyStopping(
            monitor="val_loss",
            patience=CFG.head_only_epochs,
            restore_best_weights=True,
            verbose=1,
        ),
        ModelCheckpoint(
            filepath=str(ckpt_dir / "best_model_head_only.weights.h5"),
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True,
            verbose=1,
        ),
    ],
)

# Stage 2: unfreeze and fine‐tune
backbone.trainable = True
for layer in backbone.layers[:-CFG.unfrozen_backbone_layers]:
    layer.trainable = False

optimizer_stage2 = AdamW(
    learning_rate=CFG.learning_rate * 0.1,
    weight_decay=CFG.weight_decay,
)
model.compile(
    optimizer=optimizer_stage2,
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="accuracy")],
)
print(f" Stage2: fine‐tuning last {CFG.unfrozen_backbone_layers} layers for {CFG.fine_tune_epochs} epochs")
history_ft = model.fit(
    train_ds,
    validation_data=valid_ds,
    initial_epoch=CFG.head_only_epochs,
    epochs=CFG.head_only_epochs + CFG.fine_tune_epochs,
    callbacks=[
        EarlyStopping(
            monitor="val_loss",
            patience=CFG.fine_tune_epochs,
            restore_best_weights=True,
            verbose=1,
        ),
        ModelCheckpoint(
            filepath=str(
                ckpt_dir / "best_model_finetuned.epoch{epoch:02d}_val{val_loss:.4f}.weights.h5"
            ),
            monitor="val_loss",
            mode="min",
            save_best_only=True,
            save_weights_only=True,
            verbose=1,
        ),
    ],
)

### Model Summary

In [ ]:
model.summary()

### Model Plot

In the model graph below, it may seem there are **four** inputs, but actually, there are **two** as discussed before. Our input consists of two parts, one for each response. However, for each input, we have `token_ids` and `padding_mask`, which makes it look like we have four inputs, but in reality, we have two inputs.

In [ ]:


import matplotlib.pyplot as plt

# 1. 從兩個 history 對象裡合併指標
loss = history_head.history['loss'] + history_ft.history['loss']
val_loss = history_head.history.get('val_loss', []) + history_ft.history.get('val_loss', [])

acc = history_head.history.get('sparse_categorical_accuracy', history_head.history.get('accuracy', [])) \
      + history_ft.history.get('sparse_categorical_accuracy', history_ft.history.get('accuracy', []))
val_acc = history_head.history.get('val_sparse_categorical_accuracy', history_head.history.get('val_accuracy', [])) \
          + history_ft.history.get('val_sparse_categorical_accuracy', history_ft.history.get('val_accuracy', []))

# 2. x 軸：總共的 epoch 
epochs = list(range(1, len(loss) + 1))

# 3. 繪圖
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 左：Loss
axes[0].plot(epochs, loss, label='Train Loss')
if val_loss:
    axes[0].plot(epochs, val_loss, label='Validation Loss')
axes[0].axvline(x=CFG.head_only_epochs + 0.5, color='gray', linestyle='--',
                label='Unfreeze Backbone')  # 标出解冻点
axes[0].set_title('Loss over Epochs')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()

# 右：Accuracy
axes[1].plot(epochs, acc, label='Train Accuracy')
if val_acc:
    axes[1].plot(epochs, val_acc, label='Validation Accuracy')
axes[1].axvline(x=CFG.head_only_epochs + 0.5, color='gray', linestyle='--',
                label='Unfreeze Backbone')
axes[1].set_title('Accuracy over Epochs')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()

## Load Best Model

After training, let's load the weight with best result to get the best performance.

In [ ]:

# 1. 指定 checkpoint 目錄
ckpt_dir = Path("checkpoints")
if not ckpt_dir.exists():
    raise FileNotFoundError(f"Checkpoint directory not found: {ckpt_dir}")

# 2. 列出所有 .weights.h5 文件（包括 head-only 、 finetuned）
weight_files = list(ckpt_dir.glob("*.weights.h5"))
if not weight_files:
    raise FileNotFoundError(f"No .weights.h5 files found in {ckpt_dir}")

# 3. 從文件名中解析出 epoch 數字；head-only 文件設為 0
def extract_epoch(fp: Path) -> int:
    m = re.search(r"epoch(\d+)", fp.name)
    return int(m.group(1)) if m else 0

# 4. 選出 epoch 最大的文件
latest_file = max(weight_files, key=extract_epoch)

# 5. 加載權重
model.load_weights(str(latest_file))
print(f"Loaded weights from: {latest_file}")

# 🧪 | Prediction

In [ ]:
# Build test dataset
test_texts = test_df["options"].tolist()

test_ds = build_dataset(
    options=test_texts,
    labels=None,                     # 明确告诉它没有 labels
    batch_size=CFG.batch_size,       # 或者 min(len(test_df), CFG.batch_size)
    cache=False,                     # 不 cache
    shuffle_size=0                   # 不 shuffle
)


In [ ]:
# Make predictions using the trained model on test data
test_preds = model.predict(test_ds, verbose=1)

# 📬 | Submission

Following code will prepare the submission file.

In [ ]:

# 1. 把預測結果做成 DataFrame
preds_df = pd.DataFrame(test_preds, columns=CFG.class_names)

# 2. 合併 id
sub_df = pd.concat([test_df["id"].reset_index(drop=True), preds_df], axis=1)

# 3. （可選）加最終預測類別
sub_df["pred_class"] = preds_df.idxmax(axis=1)

# 4. 寫檔並輸出確認
sub_df.to_csv("submission.csv", index=False)
print("Saved submission.csv with columns:", sub_df.columns.tolist())
sub_df.head()

# 🔭 | Future Directions

In this notebook, we've achieved a good score with a small model and modest token length. But there's plenty of room to improve. Here's how:

1. Try bigger models like `Deberta-Base` or `Deberta-Small`, or even LLMs like `Gemma`.
2. Increase max token length to reduce loss of data.
3. Use a five-fold cross-validation and ensemble to make the model robust and get better scores.
4. Add augmentation like shuffling response orders for more robust performance.
5. Train for more epochs.
6. Tune the learning rate scheduler.

# 📌 | Reference

* [LLM Science Exam: KerasCore + KerasNLP [TPU]](https://www.kaggle.com/code/awsaf49/llm-science-exam-kerascore-kerasnlp-tpu)
* [AES 2.0: KerasNLP Starter](https://www.kaggle.com/code/awsaf49/aes-2-0-kerasnlp-starter)